# Inverse kinematics

## Case 1) Scara robot

<a title="Humanrobo, CC BY-SA 3.0 &lt;https://creativecommons.org/licenses/by-sa/3.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:TI_S404-01.jpg"><img width="512" alt="TI S404-01" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/TI_S404-01.jpg/512px-TI_S404-01.jpg"></a>

The robot consists of a base and three links. The two first degrees of freedom are rotatins about vertical axes, and the last degree of freedom is a prismatic joint that determines the height of the tool-tip. 

The kinematics of the first two degrees of freedom are equivalent to this planar robot:
<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/2d-2dof-revolute.png?raw=true" width=512 />


In [2]:
# Install spatial math toolbox
!pip install spatialmath-python

     |████████████████████████████████| 162 kB 1.0 MB/s            
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for spatialmath-python, since package 'wheel' is not installed.
Using legacy 'setup.py install' for ansitable, since package 'wheel' is not installed.
Using legacy 'setup.py install' for colored, since package 'wheel' is not installed.
    Running setup.py install for colored ... done
    Running setup.py install for ansitable ... done
    Running setup.py install for spatialmath-python ... done
You should consider upgrading via the '/home/kjartan/.pyenv/versions/3.8.11/envs/tug/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import doctest
import spatialmath as sm
import matplotlib.pyplot as plt
%matplotlib notebook 

In [9]:
g1 = sm.SO3.Rz(np.pi/4)
g2 = sm.SO3.Exp([0,0,np.pi/4])
g1, g2

(   0.7071   -0.7071    0         
   0.7071    0.7071    0         
   0         0         1         
,
    0.7071   -0.7071    0         
   0.7071    0.7071    0         
   0         0         1         
)

In [16]:
def fk_scara(th, l0=1, l1=1, l2=1):
    '''
    Forward kinematics map of the scara robot
    
    Arguments
    ---------
    th : array-like of length 3
       The two joint angles and the displacement of the final prismatic joint
    l0 : float
       Height of the base
    l1, l2 : float
       Lengths of link 1 and 2, respectively
    
    Returns
    -------
    tpc : numpy array of length 3
       The position of the tool point center.
       
    Tests
    -----
    
    1) Tool tip at default position
    >>> tpc = fk_scara((0, 0, 0))
    >>> "(%0.2f, %0.2f, %0.2f)" %tuple(tpc)
    '(2.00, 0.00, 1.00)'
    
    2) Tool tip at 90 degrees in first two joints and 0.1 in last
    >>> tpc = fk_scara((np.pi/2, np.pi/2, 0.1))
    >>> "(%0.2f, %0.2f, %0.2f)" %tuple(tpc)
    '(-1.00, 1.00, 0.90)'
    
    4) Tool tip position is always inside a circle of a certain radius
    >>> positions = [fk_scara((th1_, th2_, 0))
    ... for th1_ in np.arange(0, 2*np.pi, 0.2) 
    ... for th2_ in np.arange(0, 2*np.pi, 0.2)]
    >>> distances = np.array([np.sqrt(p_[0]**2 + p_[1]**2) for p_ in positions])
    >>> max_radius = 2 + 1e-12 # Add a small tolerance
    >>> np.any(distances > max_radius)
    False
    
    '''
    
    # Center and direction of the joints 
    w = np.array([0,0,1]) # Same for all
    q1 = np.array([0,0,l0])
    q2 = np.array([l1,0,l0])
    
    # Translational parts 
    v1 = -np.cross(w, q1)
    v2 = -np.cross(w, q2)
    
    # Check sanity. The translational part of the twist is v = - w x q
    # which is the velocity of a point at the origin when undergoing the 
    # rotation about axis of direction w and axis going through q.
    assert(np.allclose(v1, [0,0,0]))
    assert(np.allclose(v2, [0,-l1,0]))
    
        # The twists
    xi1 = sm.Twist3(np.hstack((v1, w))*th[0])
    xi2 = sm.Twist3(np.hstack((v2, w))*th[1])
    xi3 = sm.Twist3(np.hstack((-w, np.zeros(3)))*th[2])
    
    # Initial position of the tool tip
    tpc0 = np.array([l1+l2, 0, l0])
    
    tpc = xi1.SE3() * xi2.SE3() * xi3.SE3() * tpc0
    return tpc[:3]
    
    

In [18]:
# Doctests for forward kinematics functions
doctest.run_docstring_examples(fk_scara, globals())

In [19]:
def ik_scara(tpc, l0=1, l1=1, l2=1):
    '''
    Inverse kinematics of a Scara robot. Using the robotics toolbox
    
    The fixed frame of the base has vertical z-axis. 
    In the default configuration the arm is completely stretched out along the base x-axis,
    and the end link is retracted so that the tool tip center is at the height of the 
    base, that is l0.
    
    Arguments
    ---------
    tpc : numpy array of length 3
       The desired tool point center
    l0 : float
       The height of the base.
    l1, l2 : float
       The length of link 1 and 2, respectively.
       
    Returns
    -------
    theta1, theta2, theta3: floats
       The three joint angles. If the point is not reachable, an exception is raised.
       
    Tests
    -----
    
    1) 
    Tool tip at its default location
    >>> tpc = np.array([2,0,1])
    >>> (th1, th2, th3) = ik_scara(tpc)
    >>> "(%0.2f, %0.2f, %0.2f)" %(th1, th2, th3)
    '(0.00, 0.00, 0.00)'
    
    2) 
    Tool tip at (-1,1,0)
    >>> tpc = np.array([-1,1,0])
    >>> (th1, th2, th3) = ik_scara(tpc)
    >>> "(%0.2f, %0.2f, %0.2f)" %(th1, th2, th3)
    '(1.57, 1.57, 1.00)'
    
    3) 
    Using the forward kinematics map, test 4 random locations. Due to the ambiguity
    of the inverse kinematics solution, compare tpc position
    >>> for i in range(4):
    ...    qs = np.random.randn(3)
    ...    tpc = fk_scara(qs)
    ...    q1, q2, q3 = ik_scara(tpc)
    ...    tpc1 = fk_scara((q1, q2, q3))
    ...    np.allclose(tpc, tpc1)
    ...
    True
    True
    True
    True

    4)
    Tool tip position out of reach of the arm
    >>> tpc = np.array([4,0,1])
    >>> (th1, th2, th3) = ik_scara(tpc)
    Traceback (most recent call last):
    ValueError: Point not reachable

'''
    
    
    x,y,z = tpc
    
    # Check that point is reachable. Assuming no limits on the final prismatic joint,
    # so the reachable space is a cylinder, possibly hollow.
    d2 = x**2 + y**2
    maxreach2 = (l1+l2)**2
    minreach2 = (l1-l2)**2
    if d2 > maxreach2 or d2 < minreach2:
        raise ValueError("Point not reachable")

    theta3 = l0 - tpc[2]
    
    costheta2 = (d2 - l1**2 - l2**2)/(2*l1*l2)
    theta2 = np.arccos(costheta2)
    
    psi = np.arccos((l1+l2*costheta2)/np.sqrt(d2))
    alpha = np.arctan2(y, x)
    theta1 = alpha - psi
    
    return theta1, theta2, theta3

## Run doctests

If tests pass, no output is generated.
